In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.vecm as vecm

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df_nzgas.csv'
df = pd.read_csv(url)
df["DATE"] = pd.to_datetime(df["DATE"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

df2 = df[(df['DATE'] >= '2009-3-1') & (df['DATE'] <= '2022-6-10')]
df3 = df[(df['DATE'] < '2009-3-1')]

In [ ]:
## lag selection 

lag_order1 = vecm.select_order(data=df[['NZBTU_USD', 'HH' ]], maxlags=6, deterministic="ci")

lag_order2 = vecm.select_order(data=df2[['NZBTU_USD', 'HH' ]], maxlags=6, deterministic="ci")

lag_order3 = vecm.select_order(data=df3[['NZBTU_USD', 'HH' ]], maxlags=6, deterministic="ci")

print(lag_order1.selected_orders, lag_order2.selected_orders, lag_order3.selected_orders)

In [2]:
## full data 

m1 = vecm.VECM(df[['NZBTU_USD', 'HH', 'NBP_USD'  ]], deterministic="ci", k_ar_diff=1, coint_rank=1)  
vecm_res_1 = m1.fit()

## before shale 
m2 = vecm.VECM(df2[['NZBTU_USD', 'HH', 'NBP_USD' ]], deterministic="ci", k_ar_diff=1, coint_rank=1)  
vecm_res_2 = m2.fit()

## after shale 
m3 = vecm.VECM(df3[['NZBTU_USD', 'HH', 'NBP_USD' ]], deterministic="ci", k_ar_diff=1, coint_rank=1)  
vecm_res_3 = m3.fit()

/Users/joe/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


In [3]:
print(vecm_res_3.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation NZBTU_USD
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
L1.NZBTU_USD     0.2310      0.174      1.331      0.183      -0.109       0.571
L1.HH            0.0605      0.025      2.394      0.017       0.011       0.110
L1.NBP_USD      -0.0618      0.034     -1.829      0.067      -0.128       0.004
Det. terms outside the coint. relation & lagged endog. parameters for equation HH
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
L1.NZBTU_USD    -2.0134      1.169     -1.722      0.085      -4.305       0.278
L1.HH            0.0419      0.170      0.246      0.805      -0.291       0.375
L1.NBP_USD       0.1945      0.228      0.855      0.393      -0.252       0.640
Det. terms outside 

In [4]:
granger_results = vecm_res_3.test_granger_causality(caused="NZBTU_USD", signif=0.05)
granger_results.summary()

Test statistic,Critical value,p-value,df
7.596,2.484,0.000,"(4, 81)"
